In [2]:
from langchain_qwq import ChatQwen
import os

model = ChatQwen(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    temperature=0.7,
)

消息有一个content属性，可以理解为发送给模型的数据有效载荷，该属性是松散类型的（可以接收字符串、列表、字典等格式的数据），这使得LangChain聊天模型能够直接支持特定于提供商的结构，例如多模态内容和其他数据。

# 标准内容块

标准内容块是LangChain提供的一种适用于所有提供商的标准消息内容表示方式。消息对象实现了一个content_blocks属性，该属性会将content属性“惰性解析”为一种标准的、类型安全的表示形式。比如ChatAnthropic 或 ChatOpenAI 生成的消息会包含各自格式的“思考”或“推理”块，但它们可以被惰性解析为一致的ReasoningContentBlock表示形式。

>“惰性解析”（Lazy Parsing）是一种“按需计算”的策略。简单来说，就是“不到万不得已，绝不干活”。模型返回的数据最初可能是一个复杂的JSON字符串或字典，但系统不会立即将其转换成LangChain内部的标准对象，只有当你真正去调用 message.content_blocks 这个属性时，系统才会开始执行转换代码，把原始数据解析成标准格式。

In [4]:
from langchain.messages import AIMessage

message = AIMessage(
    content=[
        {
            "type": "reasoning",
            "id": "rs_abc123",
            "summary": [
                {"type": "summary_text", "text": "summary 1"},
                {"type": "summary_text", "text": "summary 2"},
            ],
        },
        {"type": "text", "text": "...", "id": "msg_abc123"},
    ],
    response_metadata={"model_provider": "openai"}
)
message.content_blocks

[{'type': 'reasoning', 'id': 'rs_abc123', 'reasoning': 'summary 1'},
 {'type': 'reasoning', 'id': 'rs_abc123', 'reasoning': 'summary 2'},
 {'type': 'text', 'text': '...', 'id': 'msg_abc123'}]

如果将LC_OUTPUT_VERSION环境变量设置为v1，或者在初始化任何聊天模型时，将其output_version参数设置为 "v1"，content与content_blocks属性将返回相同的结果，langchain自动在后台完成了这个转化。

In [5]:
model = ChatQwen(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    temperature=0.7,
    output_version="v1",
)

In [ ]:
from langchain.messages import HumanMessage
response = model.invoke([HumanMessage(content="介绍一下你自己")])

print("content:", response.content)
print("="*60)
print("content_blocks:", response.content_blocks)

# 多模态内容

多模态是指大模型能够处理多种形式数据（如文本、音频、图像和视频）的能力。LangChain包含了适用于所有提供商的标准数据类型。聊天模型既可以接受多模态数据作为输入，也可以生成多模态数据作为输出。